In [77]:
import tensorflow as tf
print(tf.__version__)

2.16.1


In [78]:
from tensorflow.keras import Layer
from keras.api.models import Sequential

In [79]:
class CustomDenseLayer(Layer):
    def __init__(self, units=32):
        super(CustomDenseLayer, self).__init__()
        self.units = units
    def build(self, input_shape):
        self.w = self.add_weight(shape=(input_shape[-1], self.units),
                                 initializer = 'random_normal',
                                 trainable=True)
        self.b = self.add_weight(shape=(self.units,),
                                 initializer = 'zeros',
                                 trainable=True)
    def call(self, inputs):
        return tf.nn.relu(tf.matmul(inputs, self.w) + self.b)

In [80]:
from tensorflow.keras.layers import Dense

model = Sequential([
    CustomDenseLayer(128),
    CustomDenseLayer(32),
    CustomDenseLayer(8),
    Dense(1, activation='sigmoid')
])

In [81]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [82]:
import pandas as pd
import numpy as np

df = pd.read_csv('./heartattackprediction_classification.csv')
print(df.shape)

(8763, 26)


In [83]:
print(df.columns)

Index(['Patient ID', 'Age', 'Sex', 'Cholesterol', 'Blood Pressure',
       'Heart Rate', 'Diabetes', 'Family History', 'Smoking', 'Obesity',
       'Alcohol Consumption', 'Exercise Hours Per Week', 'Diet',
       'Previous Heart Problems', 'Medication Use', 'Stress Level',
       'Sedentary Hours Per Day', 'Income', 'BMI', 'Triglycerides',
       'Physical Activity Days Per Week', 'Sleep Hours Per Day', 'Country',
       'Continent', 'Hemisphere', 'Heart Attack Risk'],
      dtype='object')


In [84]:
x = df.iloc[:, 1:-1]
y = df.iloc[:, -1]

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
x['Sex'] = le.fit_transform(x['Sex'])
x['Diet'] = le.fit_transform(x['Diet'])
x['Country'] = le.fit_transform(x['Country'])
x['Continent'] = le.fit_transform(x['Continent'])
x['Hemisphere'] = le.fit_transform(x['Hemisphere'])
x[['Systolic', 'Diastolic']] = x['Blood Pressure'].str.split('/', expand=True)
x.drop(columns=['Blood Pressure'], inplace=True)

In [85]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)

In [86]:
x_train

,Age,Sex,Cholesterol,Heart Rate,Diabetes,Family History,Smoking,Obesity,Alcohol Consumption,Exercise Hours Per Week,...,Income,BMI,Triglycerides,Physical Activity Days Per Week,Sleep Hours Per Day,Country,Continent,Hemisphere,Systolic,Diastolic
2251,78,0,315,45,1,0,1,1,1,8.054107,...,276094,28.170032,165,6,9,9,3,1,161,91
3662,84,1,347,99,1,0,1,0,0,2.829251,...,116849,37.112647,35,6,9,11,2,1,127,101
208,24,0,152,44,1,1,0,0,1,15.482946,...,50437,37.565052,521,5,8,3,4,0,103,82
2227,50,1,227,95,0,0,1,1,0,0.733570,...,132653,22.253203,34,5,8,5,5,0,120,82
798,22,1,163,47,1,0,1,0,1,10.073059,...,234255,25.662052,737,3,4,6,3,0,104,90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5734,48,1,334,43,1,0,1,1,0,15.423465,...,59858,33.549888,549,4,4,13,0,1,94,82
5191,68,0,188,69,0,0,1,0,1,11.689453,...,60332,26.142883,410,2,9,16,1,0,116,70
5390,32,0,169,84,1,0,0,1,0,11.639443,...,67604,39.512660,777,4,9,12,0,0,156,79
860,85,0,285,54,1,1,1,1,1,11.070968,...,138980,25.499506,552,1,10,12,0,0,165,73


In [87]:
model.build((None, x_train.shape[1]))

In [88]:
model.fit(x_train, y_train, epochs = 10, batch_size=32)

Epoch 1/10
206/206 ━━━━━━━━━━━━━━━━━━━━ 1s 585us/step - accuracy: 0.5701 - loss: 17.5694
Epoch 2/10
206/206 ━━━━━━━━━━━━━━━━━━━━ 0s 578us/step - accuracy: 0.5318 - loss: 5.1953
Epoch 3/10
206/206 ━━━━━━━━━━━━━━━━━━━━ 0s 558us/step - accuracy: 0.5442 - loss: 3.4945
Epoch 4/10
206/206 ━━━━━━━━━━━━━━━━━━━━ 0s 591us/step - accuracy: 0.5655 - loss: 1.1087
Epoch 5/10
206/206 ━━━━━━━━━━━━━━━━━━━━ 0s 563us/step - accuracy: 0.5590 - loss: 1.0789
Epoch 6/10
206/206 ━━━━━━━━━━━━━━━━━━━━ 0s 616us/step - accuracy: 0.5566 - loss: 1.3983
Epoch 7/10
206/206 ━━━━━━━━━━━━━━━━━━━━ 0s 553us/step - accuracy: 0.5671 - loss: 0.8906
Epoch 8/10
206/206 ━━━━━━━━━━━━━━━━━━━━ 0s 581us/step - accuracy: 0.5701 - loss: 0.9930
Epoch 9/10
206/206 ━━━━━━━━━━━━━━━━━━━━ 0s 579us/step - accuracy: 0.5686 - loss: 0.8503
Epoch 10/10
206/206 ━━━━━━━━━━━━━━━━━━━━ 0s 640us/step - accuracy: 0.5888 - loss: 0.7179


In [89]:
loss, accuracy = model.evaluate(x_test, y_test, batch_size=32)

69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 470us/step - accuracy: 0.6429 - loss: 0.6545


In [90]:
print(accuracy)

0.6458238363265991
